# Download data and process interferogram stack
George Brencher

## Create and configure environment

In [ ]:
#Skip if environment is already created. 
#!mamba env create -f environment.yml
#!conda activate isce2
#!python -m ipykernel install --user --name=isce2

Make sure kernel is set to 'Python [conda env:isce2]'

In [ ]:
# Note--if processing on headless server, need to run the following for the stack processor to work: 
!pip install opencv-python-headless

In [ ]:
# Import required packages
import logging
log = logging.getLogger()
log.setLevel(logging.WARN)
import os
import getpass
import asf_search as asf
import isce
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
import rasterio.plot
from rasterio import logging
from osgeo import gdal

'topStack' is a Sentinel 1 interferogram stack processing workflow which has been contributed to isce. To get access to topsStack, which does not come with the conda install of ISCE, we need to clone isce2 from github. 

In [ ]:
# skip if already cloned
#!git clone https://github.com/isce-framework/isce2.git

In [ ]:
# Change to work directory 
os.chdir('/home/jovyan/Friendly-InSAR-time-series')

In [ ]:
# Set environment variables so that you can call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['PATH']+='{ISCE_HOME}/bin:{ISCE_HOME}/applications'.format(**os.environ)

# Set path to topsStack workflow in ISCE source directory
CWD = os.getcwd()
os.environ['PATH']+=f':{CWD}/isce2/contrib/stack/topsStack'
print(os.environ['PATH'])

In [ ]:
# Make sure stack processing scripts are in path
!which stackSentinel.py

In [ ]:
# Do downloads and processing in a temporary folder to avoid clogging the disk
os.chdir('/tmp')

In [ ]:
# make required directories
dirs = ['/tmp/orbital', '/tmp/SLC', '/tmp/DEM', '/tmp/AUX']
for i in dirs:
    if not os.path.exists(i):
        os.makedirs(i)

listdir('/tmp')

## Download scenes

In [ ]:
#check disk space prior to downloading SLCs
!df -h

In [ ]:
# Change to SLC directory 
os.chdir('/tmp/SLC')

In [ ]:
# Update with NASA Earthdata login to download SLC data
EARTHDATA_LOGIN = 'qbrencherUW'
EARTHDATA_PASSWORD = getpass.getpass()

# prevent DEBUG messages
logging.getLogger('urllib3').setLevel(logging.WARNING)

try:
    user_pass_session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

In [ ]:
# Create a persistent .netrc file for downloading NASA datasets (ISCE retrieves SRTM by default)
# https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget
!echo "machine urs.earthdata.nasa.gov login {EARTHDATA_LOGIN} password {EARTHDATA_PASSWORD}" > ~/.netrc
!chmod 0600 ~/.netrc

In [ ]:
# Set bounding box (larger than processing extent by at least a degree) 
# Use southwest and northeast corner points of a box in [lat, lon] format
# Must use integers

# southwestern corner
lower_left = [-2, -92]

# northeastern corner
upper_right = [0, -90]

bbox = f'POLYGON(({lower_left[1]} {lower_left[0]}, {lower_left[1]} {upper_right[0]},{upper_right[1]} {upper_right[0]}, {upper_right[1]} {lower_left[0]}, {lower_left[1]} {lower_left[0]}))'

In [ ]:
# Set platform, season, start and end dates, flight direction, and processing level.
# May want to include frame if aoi is in scene overlap area and disk space is limited
opts = {'platform': asf.PLATFORM.SENTINEL1,
        'season': '152,274',
        'start': '2021-08-01T00:00:00Z', 
        'end': '2021-09-01T00:00:00Z', 
        'flightDirection': 'D',
        'processingLevel': 'SLC',
        'frame': 593
       }

In [ ]:
# search for scenes
results = asf.search(intersectsWith=bbox, **opts)
print(f'{len(results)} scenes found')

In [ ]:
# Download scenes 
#Each is ~4 GB
results.download(path = '/tmp/SLC', session=user_pass_session)

In [ ]:
# Create a list of zipped SLCs. Assume there are no other files with .zip extension in SLC directory
tmp = listdir('/tmp/SLC')

scenes = []
for file in tmp:
    if file[-4:]=='.zip':
        scenes.append(file[:-4])
        
print(scenes)

## Download precise orbitals 

In [ ]:
os.chdir('/tmp/orbital')

In [ ]:
!which fetchOrbit.py

In [ ]:
# grab orbital files with fetchOrbit.py
for file in scenes:
    os.system(f'fetchOrbit.py -i {file}')

### Also download calibration auxliary file
A calibration auxliary (AUX_CAL) file is used for antenna pattern correction to compensate the range phase offset of SAFE products with IPF verison 002.36 (mainly for images acquired before March 2015). 

In [ ]:
os.chdir('/tmp/AUX')
!wget https://qc.sentinel1.groupcls.com/product/S1A/AUX_CAL/2014/09/08/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ

## Specify or download digital elevation file
If using your own DEM, place it in the 'DEM' directory. Must be in WGS 84. Name your DEM 'demLat.......wgs84
If you don't provide a DEM, dem.py can be used to automatically download SRTM. 

In [ ]:
os.chdir('/tmp/DEM')

In [ ]:
# make sure dem.py is in path
!which dem.py

In [ ]:
# Examine bounding box
print(f'lower left (southwest): {lower_left}, upper right (northwest): {upper_right}')

In [ ]:
# Feed dem.py ingeter coordinates surrounding the aoi, using the following structure: south lat, north lat, west lon, east lon 
os.system(f'dem.py -a stitch -b {lower_left[0]} {upper_right[0]} {lower_left[1]} {upper_right[1]} -r -s 1 -c')
!rm demLat*.dem demLat*.dem.xml demLat*.dem.vrt

In [ ]:
listdir('/tmp/DEM')

## Stack processing 

In [ ]:
os.chdir('/tmp')

In [ ]:
# Look at options for stack processing

!stackSentinel.py -h

Set some options for processing

In [ ]:
# Set number of interferograms to be made per scene
connections = 2

# Set coherence threshold for interferogram pixels
cfilter = 0.2

# Set number of looks in range. Level 1 SLC resolution in range: 2.3 m
rlooks = 9

# Set number of looks in azimuth. Level 1 SLC resolution in azimuth: 14.1 m 
zlooks = 2

# get DEM name
files = listdir('/tmp/DEM/')

for i in files:
    if i[:6] == 'demLat' and i[-5:] == 'wgs84':
        dem = i

print(f'number of connections: {connections}')
print(f'coherence threshold: {cfilter}')
print(f'looks in range: {rlooks}')
print(f'looks in azimuth: {zlooks}')
print(f'DEM name: {dem}')

In [ ]:
# generate run files
# Set processing aoi here: south limit, north limit, west limit, east limit
!stackSentinel.py -s /tmp/SLC/ -o /tmp/orbital/ -a /tmp/AUX/ -d /tmp/DEM/$dem -r $rlooks -z $zlooks -c $connections -b '-0.6 -0.2 -91.8 -91.25' -f $cfilter

In [ ]:
os.chdir('/tmp/run_files')

In [ ]:
# For an in-depth explanation of processing steps, see: https://github.com/isce-framework/isce2/blob/main/contrib/stack/topsStack/README.md
# and https://ieeexplore.ieee.org/abstract/document/7637021

In [ ]:
%%time
# 20-40 min for 3 scenes
!sh run_01_unpack_topo_reference

In [ ]:
# Plot some burst geometry files
os.chdir('/tmp/geom_reference/IW1')

# Set which burst to look at 
burst = 2 
geom_files = [f'hgt_0{burst}.rdr.vrt', f'incLocal_0{burst}.rdr.vrt', f'lat_0{burst}.rdr.vrt', f'lon_0{burst}.rdr.vrt', f'los_0{burst}.rdr.vrt', f'shadowMask_0{burst}.rdr.vrt']

In [ ]:
log = logging.getLogger()
log.setLevel(logging.WARN)

for file in geom_files:
    with rasterio.open(file) as src:
        geom = src.read(1)

    f, ax = plt.subplots(figsize=(15,5))
    im0 = ax.imshow(geom);
    f.colorbar(im0, ax=ax)
    ax.set_title(f'{file}')
    
os.chdir('/tmp/run_files')

In [ ]:
%%time
# Fast step
!sh run_02_unpack_secondary_slc

In [ ]:
%%time
# Fast step
!sh run_03_average_baseline

In [ ]:
%%time
# Fast step
!sh run_04_extract_burst_overlaps

In [ ]:
%%time
# ~2 min for 3 scenes
!sh run_05_overlap_geo2rdr

In [ ]:
%%time
# ~8 min for 3 scenes
!sh run_06_overlap_resample

In [ ]:
%%time
# ~13 min for 3 scenes
!sh run_07_pairs_misreg

In [ ]:
%%time
# Fast step
!sh run_08_timeseries_misreg

In [ ]:
%%time
# ~13 min for 3 scenes
!sh run_09_fullBurst_geo2rdr

In [ ]:
%%time
# ~9 min for 3 scenes
!sh run_10_fullBurst_resample

In [ ]:
%%time
# Fast step
!sh run_11_extract_stack_valid_region

In [ ]:
%%time
# STEP CURRENTLY BROKEN
# Fails to merge geometry files
!sh run_12_merge_reference_secondary_slc

In [ ]:
# Plot merged geometry files
####### CURRENTLY BROKEN, ISCE IS FAILING TO GENERATE MERGED GEOMETRY FILES ########
os.chdir('/tmp/merged/geom_reference/')
 
geom_files = ['hgt.rdr.vrt', 'incLocal.rdr.vrt', 'lat.rdr.vrt', 'lon.rdr.vrt', 'los.rdr.vrt', 'shadowMask.rdr.vrt']

In [ ]:
#log = logging.getLogger()
#log.setLevel(logging.WARN)

for file in geom_files:
   with rasterio.open(file) as src:
    geom = src.read(1)

    f, ax = plt.subplots(figsize=(15,5))
    im0 = ax.imshow(geom);
    f.colorbar(im0, ax=ax)
    ax.set_title(f'{file}')
    
os.chdir('/tmp/run_files')

In [ ]:
%%time
# ~3 min for 3 scenes
!sh run_13_generate_burst_igram

In [ ]:
%%time
# Fast step
!sh run_14_merge_burst_igram

In [ ]:
%%time
# ~7 min for 3 scenes
!sh run_15_filter_coherence

In [ ]:
%%time
# ~40 min to 1 hour+ for 3 scenes 
!sh run_16_unwrap

## Sanity check: plotting results

In [ ]:
# change to /merged/interferograms, where merged interferograms live
os.chdir('/tmp/merged/interferograms/')
tmp = listdir('/tmp/merged/interferograms/')

# create list of merged interferograms
int_list = []
for i in tmp:
    if len(i) == 17 and i[8]=='_':
        int_list.append(i)

print(int_list)

In [ ]:
# Here, only plotting one interferogram
int_list=['20210802_20210814']

In [ ]:
# Plot unwrapped interferograms
log = logging.getLogger()
log.setLevel(logging.WARN)

# loop to plot all interferograms
# INTERFEROGRAMS ARE NOT GEOCODED, so files will be upside down and backwards
for i in int_list:
    os.chdir(f'/tmp/merged/interferograms/{i}')
    with rasterio.open('filt_fine.unw.vrt') as src:
        amp = src.read(1)
        phase = src.read(2)

    f, ax = plt.subplots(1, 2, figsize=(15, 5))
    im0 = ax[0].imshow(np.log(amp), vmin=0);
    f.colorbar(im0, ax=ax[0])
    ax[0].set_title(f'{i} amplitude')
    ax[0].set_ylim(1500, 2700)
    ax[0].set_xlim(1700, 3000)
    
    
    im1 = ax[1].imshow(phase+17, cmap ='RdBu', vmin=-15, vmax=15)
    f.colorbar(im1, ax=ax[1])
    ax[1].set_title(f'{i} unwrapped phase')
    ax[1].set_ylim(1500, 2700)
    ax[1].set_xlim(1700, 3000)
    
    f.set_tight_layout(True)

In [ ]:
# Plot coherence
for i in int_list:
    os.chdir(f'/tmp/merged/interferograms/{i}')
    with rasterio.open('fine.cor.vrt') as src:
        coherence = src.read(2)

    fig, ax = plt.subplots(figsize=(8,5))
    im0 = ax.imshow(coherence);
    f.colorbar(im0, ax=ax)
    ax.set_title(f'{i} coherence')
    ax.set_ylim(1500, 2700)
    ax.set_xlim(1700, 3000)